# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyD-


In [3]:
openai = OpenAI()
MODEL = "gpt-4o-mini"
gemini_openai = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [4]:
def evaluate_math_expression(my_code):
    print(f"EXECUTING FUNCTION WITH CODE: {my_code}")
    exec(my_code)
    r = locals()['interpreter_result'] 
    return r


math_function = {
    "name": "evaluate_math_expression",
    "description": "Give the result of a math expression. \
    Call this whenever you need to know the result of a mathematical expression. \
    Generate python code ALWAYS with the final result assigned to a variable called 'interpreter_result'. \
    For example when a user asks 'What is 2+2' generate 'interpreter_result = 2+2', and pass this code to the tool. \
    Another example if a user ask 'What is log(5)' generate 'import math; interpreter_result = math.log(5)' and pass this code to the tool.",
    
    "parameters": {
        "type": "object",
        "properties": {
            "my_code": {
                "type": "string",
                "description": "The python math expression to evaluate",
            },
        },
        "required": ["my_code"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": math_function}]

In [5]:
evaluate_math_expression("import math; interpreter_result = math.log(5)")

EXECUTING FUNCTION WITH CODE: import math; interpreter_result = math.log(5)


1.6094379124341003

In [6]:
system_message = "You are a math assistant. \
Generate python code to give result of a math expression, always name the result 'interpreter_result'. \
For example when a user asks 'What is 2+2', generate 'interpreter_result = 2+2' and pass this code to the tool. \
Another example: if a user ask 'What is log(5)' generate 'import math; interpreter_result = math.log(5)'"

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        response = handle_tool_call(message)
        print(response)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content


def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    my_code = arguments.get('my_code')
    interpreter_result = evaluate_math_expression(my_code)
    response = {
        "role": "tool",
        "content": json.dumps({"my_code": my_code,"interpreter_result": interpreter_result}),
        "tool_call_id": tool_call.id
    }
    return response

In [7]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OHYkSo0W2zyYkHuwC4qgOEvp', function=Function(arguments='{"my_code":"interpreter_result = 2+2"}', name='evaluate_math_expression'), type='function')])
EXECUTING FUNCTION WITH CODE: interpreter_result = 2+2
{'role': 'tool', 'content': '{"my_code": "interpreter_result = 2+2", "interpreter_result": 4}', 'tool_call_id': 'call_OHYkSo0W2zyYkHuwC4qgOEvp'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gS8lsyJjtbiC7oypgovSVwbV', function=Function(arguments='{"my_code":"import math; interpreter_result = 5 - 9 + math.exp(6)"}', name='evaluate_math_expression'), type='function')])
EXECUTING FUNCTION WITH CODE: import math; interpreter_result = 5 - 9 + math.exp(6)
{'role': 'tool', 'content': '{"my_code": "import math; interpreter_result = 5 - 9 + m